# Redback: an open source bayesian inference package for fitting electromagnetic transients. 

## How redback can be useful to you.
- ### Download data for supernovae, tidal disruption events, gamma-ray burst afterglows, kilonovae, prompt emission from different catalogs/telescopes; Swift, BATSE, Open access catalogs. Users can also provide their own data or use simulated data

- ### Redback processes the data into a homogeneous transient object, plotting lightcurves and doing other processing.

- ### The user can then fit one of the models implemented in redback. Or fit their own model. Models for several different types of electromagnetic transients are implemented and range from simple analytical models to numerical surrogates.

- ### All models are implemented as functions and can be used to simulate populations, without needing to provide data. This way redback can be used simply as a tool to simulate realistic populations, no need to actually fit anything.

- ### [Bilby](https://lscsoft.docs.ligo.org/bilby/index.html) under the hood. Can easily switch samplers/likelihoods etc. Over 15 samplers are implemented and the list continues to grow. 

- ### Fitting returns a homogenous result object, with functionality to plot lightcurves/walkers/corner and the posterior/evidence/credible interval etc. This way redback results can feed into hierarchical analysis of populations of transients or be used in reweighting.

### Online documentation

- ### [Installation](https://redback.readthedocs.io/en/latest/)
- ### [Examples](https://github.com/nikhil-sarin/redback/tree/master/examples)
- ### [Documentation](https://redback.readthedocs.io/en/latest/)

### Contributing 
- ### Redback is currently in alpha with a paper in preparation. If you are interested in contributing please join the redback [slack](https://join.slack.com/t/slack-23u4492/shared_invite/zt-14y9q1qmo-VRmc8ZxHzB3u~dB3Wi6pjw) and get in touch with [me](mailto:nikhil.sarin@su.se?subject=Contributing%20to%20redback). 
- ### All contributors at the alpha stage will be invited to be co-authors of the first paper.

<!-- ![](notebook_images/RedbackLogo.png) -->
<img src="notebook_images/RedbackLogo.png" alt="drawing" width="500"/>

In [ ]:
import redback 
import pandas as pd
from bilby.core.prior import PriorDict
import bilby
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>.output { max-width:100% !important; }</style>"))
display(HTML("""<style>.output { display: flex; align-items: center;text-align: center;}</style>"""))
%pylab inline

# Downloading data from different catalogs e.g., the open access catalogs

In [ ]:
kne = 'at2017gfo'

data = redback.get_data.get_kilonova_data_from_open_transient_catalog_data(transient=kne)
data

### The user is returned the data in a pandas dataframe. The data is also saved in a sensible way. There are two files; the raw data file and the processed data file where we do some basic processing and make it homogenous. Note that the data returned here is a simple pandas dataframe and can be manipulated in arbitrary ways.

### For example, let's say I just want to see the i band data

In [ ]:
data[data['band']=='i']

## Can similarly download data from LASAIR (ZTF/Vera Rubin broker), Afterglows from Swift, Prompt grb from Swift and BATSE.  

In [ ]:
GRB = '070809'
# Flux density, flux data
data = redback.get_data.get_bat_xrt_afterglow_data_from_swift(grb=GRB, data_mode="flux")
data

In [ ]:
sne = "SN2011kl"
data = redback.get_data.get_supernova_data_from_open_transient_catalog_data(sne)
data

In [ ]:
data = redback.get_data.get_lasair_data(transient="ZTF19aagqkrq", transient_type="afterglow")
data

# Since this data is just a simple dataframe, you could play around with it yourself to create plots etc. However, we provide functionality to load this data into a 'transient' object. Providing methods for plotting and other functionality. 

## There are 7 different types of transient objects implemented in redback, which all have unique functionality for the specific type of transient. 

### There are two parent classes

- Transient: For any type of generic transient
- OpticalTransient: For any type of generic optical transient

### Five more targeted transient classes

- SGRB
- LGRB
- Supernova 
- Kilonova
- Tidal disruption event
- Prompt 

### These classes come with lots of functionality and lookup tables which provide metadata useful for further analysis, such as redshift, T90, start time, etc. They also allow other processing such as converting flux to luminosity. 

### For each of the transients we have different data_modes which determines what data to fit, plot labels, type of likelihood to use etc. We note that the latter two can be changed by users if desired.

- luminosity, flux, flux_density, magnitude, counts, time tagged events

## Creating a kilonova object for at2017gfo

In [ ]:
kne = 'at2017gfo'
kilonova = redback.kilonova.Kilonova.from_open_access_catalogue(
    name=kne, data_mode="magnitude", active_bands=np.array(["g", "i"]))
kilonova.plot_data(save=False, plot_others=False)

### Here we created the kilonova transient object using the open access catalogue data, with the `data mode == 'flux_density'`. 

### Here we have also specified `active_bands=np.array(['g', 'i')`, which sets the rest of the data to be inactive, i.e., not used in the fitting. All bands/frequencies are active by default.

### The function returns the axes so user can change the ylim etc from the default themselves as they would for any other matplotlib plot. Or pass it into the function as a keyword argument

In [ ]:
kilonova = redback.kilonova.Kilonova.from_open_access_catalogue(
    name=kne, data_mode="magnitude", active_bands=np.array(["g", "i", "r", "z"]))
ax = kilonova.plot_data(save=False, show=False, plot_others=True)
ax.set_ylim(25,16)
plt.show()

In [ ]:
kilonova = redback.kilonova.Kilonova.from_open_access_catalogue(
    name=kne, data_mode="magnitude", active_bands=np.array(["g", "i", "r", "z"]))
ax = kilonova.plot_data(save=False, show=False, plot_others=False, xlim_high=10)

### Many other plotting aesthetic things can just be passed into the function. 

### We also provide a simple plot_multiband method. Which will plot one band per panel.

In [ ]:
fig, axes = plt.subplots(3, 2, sharex=True, sharey=True, figsize=(12, 8))
ax = kilonova.plot_multiband(figure=fig, axes=axes,
                        filters=["g", "r", "i", "z", "y", "J"], save=False)

### Here we also passed in our own constructred figure and axes to get the exact look. If you dont pass these in redback will figure it out on its own. Again, the axes are returned so users can also tinker with the plot further. Or pass things as keyword arguments in the function.

### We can do the same thing with afterglows, supernovae, tde's etc etc

In [ ]:
afterglow = redback.afterglow.SGRB.from_swift_grb(name=GRB, data_mode='flux',
                                                  truncate=True, truncate_method="prompt_time_error")
afterglow.analytical_flux_to_luminosity()
ax = afterglow.plot_data()

## We can also plot/fit data in time in MJD, for scenario's when you don't know the true burst start time. This is done via using `use_phase_model=True`. When we get to fitting, this flag will ensure we use the right data and also infer the start time of the transient.

In [ ]:
supernova = redback.supernova.Supernova.from_open_access_catalogue(name=sne, data_mode='flux_density', 
                                                                   use_phase_model=True)
ax = supernova.plot_multiband(filters=["J", "H", "g", "i"])

# Sometimes the user may have their own data that they simulated or was their own private data. 

### All redback transient objects can be constructed by just passing in the relevant properties. Enabling the same functionality as above (and to use in fitting..)

In [ ]:
data = pd.read_csv('example_data/grb_afterglow.csv')
data['band'] = 'x'
data['band'].iloc[data['frequency'] == 2.418000e+17] = 'X-ray'
data['band'].iloc[data['frequency'] == 3.000000e+09] = 'Radio 3 GHz'
data['band'].iloc[data['frequency'] == 6.000000e+09] = 'Radio 6 GHz'
data['band'].iloc[data['frequency'] == 5.090000e+14] = 'V'
data['band'].iloc[data['frequency'] == 3.730000e+14] = 'i'
time_d = data['time'].values
flux_density = data['flux'].values
frequency = data['frequency'].values
flux_density_err = data['flux_err'].values
bands = data['band'].values

data_mode = 'flux_density'

name = '170817A'

afterglow = redback.transient.Afterglow(
    name=name, data_mode=data_mode, time=time_d,
    flux_density=flux_density, flux_density_err=flux_density_err, frequency=frequency, bands=bands)

ax = afterglow.plot_data()

### Like all other plots users can change things like plot labels, limits etc etc either by passing in a keyword argument or by modifying the axes of the plot.

In [ ]:
afterglow = redback.transient.Afterglow(
    name=name, data_mode=data_mode, time=time_d,
    flux_density=flux_density, flux_density_err=flux_density_err, frequency=frequency)

ax = afterglow.plot_multiband()

# We could also simulate the observations ourselves, then load each set of observations into a transient object and then do inference!!

## To simulate observations, we need a model.

## In redback we have already implemented a lot of different models, which can be combined or modified to create another model easily. These models range from phenomenological, to analytical, semi-analytical to numerical surrogates built with machine learning techniques. Implementing a new model is probably the easiest way to contribute to redback!

### Specifically, the models already included are

### Afterglow models:

- Several structured jet models implemented in afterglowpy.
- Tophat jet implemented in afterglowpy.
- Cocoon
- Kilonova afterglow
- Surrogate built on top of jetfit.
- Surrogate built on top of boxfit.

### Kilonova models

- One/two/three component kilonova models
- two_layer_stratified_kilonova
- power_law_stratified_kilonova
- kilonova heating rate
- One component BNS ejecta relation
- Two component BNS ejecta relation
- One component NSBH ejecta relation
- Two component NSBH ejecta relation
- Polytrope EOS ejecta relation
- Aspherical kilonova
- Three component model - ejecta relation
- Three component model - temperature floor
- Three component model - temperature floor and diffusion
- Metzger 2017
- Surrogates of several different numerical simulations e.g., Possis, Sedona
- All kilonova models in gwemlightcurves

### Supernova models

- Arnett
- CSM
- CSM + Ni
- Basic magnetar powered
- General magnetar powered
- Supernova 1A
- Supernova 1C
- SNcosmo
- magnetar + nickel
- SLSN
- exponential powerlaw
- Shock cooling + Arnett

### Shock models
- Shock cooling
- Thermal synchrotron
- Shocked cocoon
- CSM truncation shock

### Magnetar driven ejecta models

- Metzger magnetar driven kilonova
- Mergernova
- Trapped magnetar
- General magnetar driven kilonova
- Evolving magnetar magnetar driven kilonova
- Evolving magnetar mergernova

### Millisecond magnetar models

- vacuum dipole magnetar
- magnetar with variable braking index
- GW + EM magnetar
- evolving magnetar
- magnetar with radiative losses
- collapsing magnetar
- piecewise magnetar

### Tidal disruption models

- Simple analytic fallback
- Surrogate from numerical simulation

### Phenomenological/fireball models/other exotica

- Fast blue optical transients
- Skew gaussian
- Skew exponential 
- fred
- fred_extended
- Gaussian
- 1-6 component piecewise power law
- exponential_powerlaw

### We note that these models can output in flux_density or magnitude set by the keyword argument output_format or using the appropriate bolometric/flux function.

### Alongside these models we also include some general models which can many of the above models as a base_model

- Homologous expansion
- Thin shell
- Extinction models
- Phase models
- Phase + extinction models
- Gaussian process base model: Will be soon implemented.

### You can also make several modifications to all models using dependency injections or switches

### For a full up to date list of models implemented in redback, look at the [API](https://redback.readthedocs.io/en/latest/index.html)

## All models in redback are implemented as simple functions that do not require any other redback infrastructure. They can be used to simulate populations, get a sense of the impact of different parameters, or for debugging.

In [ ]:
from redback.constants import day_to_s
from redback.model_library import all_models_dict

model = 'arnett_bolometric'

function = all_models_dict[model]
time = np.logspace(2, 8, 100)/day_to_s
bolometric_luminosity = function(time, f_nickel=0.2,
                    mej=30, vej=10000, kappa=2, kappa_gamma=1e2)
plt.loglog(time, bolometric_luminosity)
plt.xlabel('Time [days]')
plt.ylabel(r'$L_{\rm bol}$')

### Every function is documented, describing what the inputs are; their units etc etc. For some models we have also implemented a simple way to get a link to the paper describing it which provides further details. 

![](notebook_images/Docs.png)

In [ ]:
print(function.citation)

## We can also simulate an entire population by creating a population prior (what distribution each of the parameters for the entire population are drawn from) and simulate lightcurves for all of them. This does not capture realistic survey features e.g., cadence but that can be easily incorporated.

### Redback uses bilby for priors and there are plenty to choose from.

Analytical priors
-------------------------

- Beta
- Categorical
- Cauchy
- ChiSquared
- Cosine
- DeltaFunction
- Exponential
- FermiDirac
- Gamma
- Gaussian
- HalfGaussian
- LogGaussian
- LogUniform
- Logistic
- Lorentzian
- PowerLaw
- Sine
- StudentT
- SymmetricLogUniform
- TruncatedGaussian
- Uniform


Interpolated or from file
-------------------------

Users can also create a prior from a grid of values i.e., an interpolated_prior.
See documentation [here](https://lscsoft.docs.ligo.org/bilby/api/bilby.core.prior.interpolated.Interped.html#bilby.core.prior.interpolated.Interped).

### Every function has a default prior which can be loaded via

In [ ]:
priors = redback.priors.get_priors(model=model)
priors

### This prior object is essentially a dictionary of the different priors describing the shape, range, latex labels and units of each of the parameters. You can overwrite any of the priors as you would a standard python dictionary

In [ ]:
priors['f_nickel'] = 0.5

### We can sample randomly from the prior to create fake lightcurves

In [ ]:
samples = priors.sample(100)
samples = pd.DataFrame(samples)
samples

### We can place complex constraints on our prior to mimic a realistic survey. Say for example I wanted to create a population where none of the population was dimmer than 24th mag at peak in the r mag and that the peak was less than 50 days.

In [ ]:
def brightness_constraint(parameters):
    """
    Ensure the Supernova is not dimmer than 24th Mag at peak and that the peak is at less than 50 days.
    """
    converted_parameters = parameters.copy()
    converted_parameters = pd.DataFrame(converted_parameters)
    kwargs = {}
    kwargs['frequency'] = redback.utils.bands_to_frequency('r')
    kwargs['output_format'] = 'magnitude'
    tdays = np.logspace(2, 7, 50)/day_to_s
    mags = np.zeros(len(converted_parameters))
    peak_t = np.zeros(len(converted_parameters))
    for x in range(len(mags)):
        mag = function(tdays, **converted_parameters.iloc[x], **kwargs)
        mags[x] = np.min(mag)
        peak_t[x] = tdays[np.argmax(mag)]
    converted_parameters['peak_constraint'] = 24 - mags
    converted_parameters['peak_time'] = 50 - peak_t
    return converted_parameters

model = 'arnett'

function = all_models_dict[model]


priors = PriorDict(conversion_function=brightness_constraint)
priors['peak_constraint'] = bilby.core.prior.Constraint(0, 10)
priors['peak_time'] = bilby.core.prior.Constraint(0, 10)
priors.update(redback.priors.get_priors(model))
priors['redshift'] = 0.5
population_samples = pd.DataFrame(priors.sample(50))

# We can now go through and create r band lightcurves for all of them. We can also similarly create light curves for any other filter. You can also use your own model as the 'engine'. 

In [ ]:
for x in range(len(population_samples)):
    tdays = np.logspace(2, 7, 50)/day_to_s
    kwargs = {}
    kwargs['frequency'] = redback.utils.bands_to_frequency('r')
    kwargs['output_format'] = 'magnitude'
    mags = function(tdays, **population_samples.iloc[x], **kwargs)
    plt.semilogx(tdays, mags, 'C0o', c='red')
plt.gca().invert_yaxis()
plt.xlabel('Time [days]')
plt.ylabel('Magnitude')
plt.xlim(1e-3,40)
plt.ylim(35, 16)

# The raw lightcurve can then be processed through redback.simulation to create realistic observations for real surveys/telescopes e.g., Rubin, Roman, ZTF or any other configuration built from user inputs.

# With stuff about data/priors out of the way. Let's now turn to the primary purpose of redback: fitting. 

## Redback workflow for fitting
- ### Download the data from a public catalog, or provide your own data, or simulate it.

- ### Load the data into a homogenous transient object, which does the necessary processing and provides simple way to plot data. This also finds other metadata such as host galaxy/redshift/sky position if available on a public database

- ### Specify a model (either already implemented in redback or their own function).

- ### Write a prior or use the default priors.
    - Every model has default priors already implemented
    - Place constraints on the prior if necessary. These could be constraints related to the region the model is physical/something about the observation/non detections (this is one way but there are others), or where a numerical surrogate is trained on etc. 

- ### Specify a sampler and sampler settings as in bilby

- ### Fit model!

- ### The fit returns a homogenous result object, which can be used for further diagnostics, and provides a simple way to plot the fit.


# With stuff about data/priors out of the way. Let's now turn to the primary purpose of redback: fitting. 

## Redback workflow for fitting
- ### Download the data from a public catalog, or provide your own data, or simulate it.

- ### Load the data into a homogenous transient object, which does the necessary processing and provides simple way to plot data. This also finds other metadata such as host galaxy/redshift/sky position if available on a public database

- ### Specify a model (either already implemented in redback or their own function).

- ### Write a prior or use the default priors.
    - Every model has default priors already implemented
    - Place constraints on the prior if necessary. These could be constraints related to the region the model is physical/something about the observation/non detections (this is one way but there are others), or where a numerical surrogate is trained on etc. 

- ### Specify a sampler and sampler settings as in bilby

- ### Fit model!

- ### The fit returns a homogenous result object, which can be used for further diagnostics, and provides a simple way to plot the fit.


## The examples provide more detailed complicated examples of fitting different transients. Here in the interest of both time and to reduce complexity; I'll show a really simple/fast example.

In [ ]:
# first specify some basic sampler settings, model name, transient name etc etc 

model = 'evolving_magnetar'
GRB = '070809'
# number of live points. Lower is faster but worse. Higher is slower but more reliable. 
nlive = 500
sampler = 'nestle'

#download the data 
data = redback.get_data.get_bat_xrt_afterglow_data_from_swift(grb=GRB, data_mode="flux")

# create the afterglow object; 
# truncate the data using the prompt_time_error method to get rid of 
# any erronous BAT data points not belonging to the transient.
afterglow = redback.afterglow.SGRB.from_swift_grb(name=GRB, data_mode='flux',
                                                  truncate=True, truncate_method="prompt_time_error")
# convert flux data to luminosity using an analytical approximation. 
# We could also use a numerical method utilising CIAO/Sherpa and the spectrum. 
afterglow.analytical_flux_to_luminosity()

# load the default priors for the model 
priors = redback.priors.get_priors(model=model)

result = redback.fit_model(model=model, sampler=sampler, nlive=nlive, transient=afterglow,
                           prior=priors, sample='rslice', resume=True)


### The fitting will print out a bunch of things which are helpful diagnostics and indicate how things are processing, what settings are used and when things will finish. Most samplers have checkpointing so if for some reason your computer crashes/supercomputer goes down; progress is not lost.

### The fitting returns a result object which has a lot of different attributes and methods allowing further diagnostics.

### A dataframe of the posterior values

In [ ]:
result.posterior

### Other metadata/methods

In [ ]:
print(result.log_evidence)
print(result.log_evidence_err)
print(result.bayesian_model_dimensionality)
print(result.covariance_matrix)
print(result.information_gain)
print(result.max_autocorrelation_time)
print(result.transient)
print(result.transient_type)
print(result.occam_factor(result.priors))

### Plotting methods

In [ ]:
result.plot_corner(parameters=['p0', 'muinf', 'mu0', 'alpha_1', 'tm'], save=False, priors=True)
plt.show()

In [ ]:
result.plot_lightcurve(random_models=100)

## Method to plot multiband lightcurve; 1 band/frequency on each panel with fit going through them.

`result.plot_multiband_lightcurve`

## Method to plot the transient data; i.e., same thing as the transient object plot_data and plot_multiband.

`result.plot_data`

`result.plot_multiband`

## Method to plot the CDF and PDF of all parameters/log_likelihood and log_prior

`result.plot_marginals`

## Method to plot the walkers if using an MCMC sampler 

`result.plot_walkers`


## Other result features

- ### Reweighting to a different prior/model.

- ### Changing formats; creating an arviz result object.

- ### Making pp plots for a population

- ### Hierarchical inference/recycling functionality.


In [ ]:
redback.utils.calc_one_dimensional_median_and_error_bar(result.posterior['alpha_1'], 
                                                        quantiles=(0.16,0.84), fmt='.2f').string

## You can also load a result file from different analyses enabling the same functionality as above.

In [ ]:
path = 'GRBData/afterglow/luminosity/evolving_magnetar/GRB070809_result.json'
my_result = redback.result.read_in_result(path)

## Contributing/Plans

### Redback is currently in alpha with a paper in prep. All contributers in alpha will be invited to be coauthors. The contribution can range from simply testing/bug fixing the package to implementing new features/models etc. 


Learning more
--------

#### There are several examples showing the different abilities of redback in more detail. 

- #### broadband_afterglow_private_data_example
- #### fit_your_own_model_example
- #### kilonova_example
- #### magnetar_boosted_example
- #### magnetar_example
- #### prompt_example
- #### supernova_example
- #### tde_example
- #### SN2011kl_sample_in_t0_example

Coming soon:
---------------- 

- #### Sampling with constraint priors
- #### homologous expansion
- #### Simulating a population with redback
- #### Creating and sampling with new ejecta relations
- #### Joint likelihood analysis
- #### Gaussian process with a redback base model using george/celerite

### Check out all the examples [here](https://github.com/nikhil-sarin/redback/tree/master/examples). There is also extensive [documentation](https://redback.readthedocs.io/en/latest/). 

<img src="notebook_images/RedbackLogo.png" alt="drawing" width="500"/>